In [8]:
import os
import numpy as np
import imageio
import cv2
from scipy.ndimage import zoom

BASE_DIR = "/home/j/FPGA-project/voxelmorph/data/image slice-T1"        
OUTPUT_DIR = "/home/j/FPGA-project/tools/calibration_dataset"  
TARGET_SHAPE = (160, 192, 224)

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [9]:
def resize_volume(vol, target_shape):
    z, y, x = vol.shape
    tz, ty, tx = target_shape

    resized_slices = [
        cv2.resize(vol[i], (tx, ty), interpolation=cv2.INTER_LINEAR)
        for i in range(z)
    ]
    stack = np.stack(resized_slices, axis=0)

    if z != tz:
        zoom_factor = (tz / z, 1.0, 1.0)
        stack = zoom(stack, zoom_factor, order=1)

    return stack


In [10]:
def load_png_volume(folder_path, target_shape):
    png_files = sorted([
        f for f in os.listdir(folder_path)
        if f.lower().endswith(".png")
    ])
    if len(png_files) == 0:
        return None

    slices = [imageio.imread(os.path.join(folder_path, f)) for f in png_files]
    volume = np.stack(slices, axis=0).astype(np.float32)

    volume = (volume - volume.min()) / (volume.max() - volume.min() + 1e-5)

    return resize_volume(volume, target_shape)


In [11]:
created = 0
rng = np.random.RandomState(42)

for subject in sorted(os.listdir(BASE_DIR)):
    subject_path = os.path.join(BASE_DIR, subject)
    if not os.path.isdir(subject_path):
        continue

    pngs = [f for f in os.listdir(subject_path) if f.lower().endswith('.png')]
    if len(pngs) == 0:
        print(f"No PNGs in {subject_path}")
        continue

    vol = load_png_volume(subject_path, TARGET_SHAPE)
    if vol is None:
        print(f"Failed to load volume from {subject_path}")
        continue

    moving = np.clip(vol + rng.normal(0, 0.05, vol.shape), 0, 1)
    fixed = vol
    pair = np.stack([moving, fixed], axis=0)  

    out_path = os.path.join(OUTPUT_DIR, f"calib_pair_{created}.npy")
    np.save(out_path, pair)

    print(f"Saved: {out_path}")
    created += 1

    if created >= 20:
        print("20 calibration pairs created.")
        break



/tmp/ipykernel_97729/1078799286.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  slices = [imageio.imread(os.path.join(folder_path, f)) for f in png_files]


Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_0.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_1.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_2.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_3.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_4.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_5.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_6.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_7.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_8.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_9.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_10.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_11.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_12.npy
Saved: /home/j/FPGA-project/tools/calibration_dataset/calib_pair_13.npy
Sa